# Julia言語と高校数学

このノートブックでは、Julia言語を使って高校数学の様々な分野を学習します。

## 1. Julia言語の概要

Juliaは統計処理や科学技術計算、機械学習に強いプログラミング言語です。
Google Colabで無料で利用できるようになりました。

公式サイト: https://julialang.org/

In [ ]:
# 必要なパッケージのインストール（初回のみ、10分程度かかります）
using Pkg
Pkg.add("Nemo")
Pkg.add("Symbolics")
Pkg.add("SimplePolynomials")
Pkg.add("Optim")
Pkg.add("Distributions")
Pkg.add("Combinatorics")
Pkg.add("StatsPlots")
using Nemo, Symbolics, SimplePolynomials, Optim, Distributions, Combinatorics, StatsPlots;

## 2. 基本的な計算

In [ ]:
# 四則演算
1 + 1

In [ ]:
# 数学定数
π , √2

In [ ]:
# 分数計算（小数と有理数の違い）
println("小数計算: ", 1/3 + 3*2/4)
println("有理数計算: ", 1//3 + 3*2//4)

In [ ]:
# 関数の定義
f(x) = x^2 + 2x - 4
println("f(0) = ", f(0))
println("f(2) = ", f(2))
println("f(-√3) = ", f(-√3))

## 3. グラフ描画

In [ ]:
using Plots
ENV["GKS_ENCODING"] = "utf8"
gr(fontfamily="IPAexGothic")  # Linux


# sin関数のグラフ
plot(sin, xlabel="x", ylabel="sin(x)", title="正弦関数")

In [ ]:
# 2次関数のグラフ
f(x) = x^2 + 2x - 4
g(x) = abs(f(x))

plot(f, -5, 3, label="f(x) = x² + 2x - 4", lw=2)
plot!(g, -5, 3, label="g(x) = |f(x)|", lw=2, ls=:dash)

In [ ]:
# 媒介変数表示（リサージュ曲線）
t = 0:0.01:2π
xs = sin.(3t)
ys = sin.(2t)

plot(xs, ys, aspectratio=true, title="リサージュ曲線", lw=2)

In [ ]:
# 3D曲面
surface(0:0.1:2π, 0:0.1:2π, (x,y)->sin(x+y), title="z = sin(x+y)")

## 4. 方程式の解法

In [ ]:
# 記号的解法
using Nemo , Symbolics

@variables x
f(x) = x^2 + 2x - 4

println("x² + 2x - 4 = 0 の解:")
symbolic_solve(f(x),x)


In [ ]:
# 解の公式の一般形

@variables a b c x
println("ax² + bx + c = 0 の解:")
symbolic_solve(a*x^2 + b*x + c, x)

In [ ]:
# 数値的解法
using SimplePolynomials

x = getx()
println("x⁵ - 1 = 0 の解（5次単位根）:")
SimplePolynomials.roots(x^5 - 1)

## 5. 漸化式と数列

In [ ]:
# 漸化式 a₁ = 3, aₙ₊₁ = 2aₙ + 1
function seq_a(n::Int)
    if n == 1
        3
    elseif n ≥ 2
        2 * seq_a(n - 1) + 1
    end
end

# 最初の10項を表示
println("数列の最初の10項:")
for i in 1:10
    println("a$i = ", seq_a(i))
end

In [ ]:
# 数列のグラフ化
using Plots 
X = 1:10
Y = seq_a.(X)

plot(X, Y, label="aₙ", lw=2)
scatter!(X, Y, label="", markersize=5)

## 6. 最適化問題

In [ ]:
# 線形計画法の可視化
# 3x + 4y ≤ 10, 4x + y ≤ 4, x ≥ 0, y ≥ 0 のとき x + y の最大値

function feasible_region(x, y)
    if 3x + 4y ≤ 10 && 4x + y ≤ 4 && x ≥ 0 && y ≥ 0
        1
    else
        0
    end
end

contour(-0.5:0.01:2, -0.5:0.01:3, feasible_region, 
        levels=[0.5], fill=true, title="実行可能領域")

In [52]:
# 非線形最適化
using Optim

# a³ + b³ - 2ab の最小値を求める（a, b > 0）
f(x) = x[1]^3 + x[2]^3 - 2*x[1]*x[2]

x0 = [1.0, 1.0]
lower = [1e-6, 1e-6]
upper = [10.0, 10.0]

result = optimize(f, lower, upper, x0)
x_opt = Optim.minimizer(result)
min_f = Optim.minimum(result)

println("最適な (a, b) = ", x_opt)
println("最小値 = ", min_f)

最適な (a, b) = [0.6666666677815241, 0.6666666677815241]
最小値 = -0.2962962962962963


## 7. 統計・確率


---

### 問題：スマートフォン利用率の調査

ある高校の生徒150名を無作為に選んで調査したところ、スマートフォンを所有している生徒は126名でした。

この高校の生徒全体におけるスマートフォン所有率（母比率 p）について、信頼度95%の信頼区間を求めなさい。

#### 与えられた情報
- 標本サイズ：n = 150名
- スマートフォン所有者数：x = 126名
- 標本比率：p̂ = 126/150 = 0.84
- 信頼度：95%（有意水準 α = 0.05）

#### 求めるもの
母比率 p の95%信頼区間

---

**解答例**：信頼区間は約 (0.7814, 0.8986) となり、この高校のスマートフォン所有率は78.1%～89.9%の間にあると95%の信頼度で推定できます。

In [ ]:
# 母比率の信頼区間（Wald法）
using Distributions

n = 150  # サンプルサイズ
x = 126  # 成功数（スマートフォン所有者）
p̂ = x / n
α = 0.05

se = sqrt(p̂ * (1 - p̂) / n)
z = cquantile(Normal(), α/2)

lower = p̂ - z * se
upper = p̂ + z * se

println("母比率の95%信頼区間: (", round(lower, digits=4), ", ", round(upper, digits=4), ")")
println("スマートフォン所有率は約", round(p̂*100, digits=1), "%と推定されます")

In [ ]:
# 正規分布と二項分布の可視化
using StatsPlots

p1 = plot(Normal(0, 1), title="標準正規分布", lw=2)
p2 = plot(Binomial(100, 1/3), title="二項分布 B(100, 1/3)", lw=2)
plot(p1, p2, layout=(1, 2), size=(800, 300))

## 8. モンテカルロ法による積分

In [ ]:
# ∫₀¹ sin(x²) dx をモンテカルロ法で計算（フレネル積分）
using Distributions

function montecarlo(f, a, b, n)
    sum(f(rand(Uniform(a, b))) for _ = 1:n) / n * (b - a)
end

f(x) = sin(x^2)
result = montecarlo(f, 0, 1, 10^6)

println("∫₀¹ sin(x²) dx ≈ ", result)
println("理論値（フレネル積分）≈ 0.3103...")

In [ ]:
# 被積分関数の可視化\n",

plot(f, 0, 1, fill=(0, 0.3, :red), label="∫₀¹ sin(x²) dx", lw=2)
plot!(x -> sin(x^2), 0, 3, label="y = sin(x²)", lw=2, color=:blue)
title!("フレネル積分")

## 9. 実践問題：場合の数と確率

In [ ]:
# 問題：1から9までの数字から3つ選んで3桁の整数を作る
using Combinatorics

X = [i for i in 1:9]
Y = permutations(X, 3) |> collect

println("(1) 全部で ", length(Y), " 通り")

# 奇数の個数
odd_count = sum(1 for y in Y if y[3] % 2 == 1)
println("(2) 奇数は ", odd_count, " 個")

In [ ]:
# 確率のシミュレーション：赤玉3個、白玉5個から2個取り出す
P = "🔴 🔴 🔴 ⚪ ⚪ ⚪ ⚪ ⚪"
Q = combinations(split(P, " "), 2) |> collect

# (1) 2個とも赤玉の確率（理論値）
prob_2red = sum(1 for q in Q if q[1] == "🔴" && q[2] == "🔴") // length(Q)
println("2個とも赤玉の確率（理論値）: ", prob_2red)

# (2) 1個赤玉、1個白玉の確率（理論値）
prob_1red1white = sum(1 for q in Q if q[1] != q[2]) // length(Q)
println("1個赤玉、1個白玉の確率（理論値）: ", prob_1red1white)

In [ ]:
# モンテカルロシミュレーション
n = 10^6
k = 0

for i = 1:n
    q = rand(Q)
    k += (q == ["🔴", "🔴"])
end

println("\n10^6回の実験による2個とも赤玉の確率: ", k/n)
println("理論値: ", float(prob_2red))

## まとめ

このノートブックでは、Julia言語を使って高校数学の様々な分野を学習しました：

1. **基本計算**: 四則演算、関数定義
2. **グラフ描画**: 2D、3D、媒介変数表示
3. **方程式**: 記号的解法と数値的解法
4. **数列**: 漸化式の実装
5. **最適化**: 線形・非線形問題
6. **統計**: 信頼区間、分布
7. **モンテカルロ法**: 数値積分、確率シミュレーション

Julia言語を使うことで、数学の抽象的な概念を具体的に可視化し、
実験的に理解を深めることができます。

＜その他のサイト＞

・[大学入試とJulia言語（JuliaTokai #21）](https://github.com/shimizudan/20250327tokyo-u?tab=readme-ov-file)


・[オンライン整数列大辞典(OEIS)に数列を登録してみた！](https://github.com/shimizudan/20250216sundaymath?tab=readme-ov-file)




＜参考サイト＞

・[Julia言語　ドキュメント（日本語翻訳版）（ごまふあざらし(GomahuAzarashi)）](https://atelierarith.github.io/UnofficialJuliaDocJP/index.html)


・[数学と物理におけるJuliaの活用](https://akio-tomiya.github.io/julia_imi_workshop2023/)

・[数学ソフトウェアとフリードキュメント XXVIII での講演資料（黒木玄）](https://github.com/genkuroki/msfd28/blob/master/README.md)

・[中心極限定理の視覚化の例（黒木玄）](https://colab.research.google.com/drive/1OKlTBuxXw_gA6oyzlk5ACInH2PaebdjC?usp=sharing)


・[数値計算法基礎 (2023)（降籏 大介）](http://www.cas.cmc.osaka-u.ac.jp/~paoon/Lectures/2023-8Semester-NA-basic/01-guide-of-julia/)



・[Juliaで学ぶ最適化と機械学習（2024）（松井勇佑）](https://matsui528.github.io/julia_opt_ml_2024/)


＜書籍＞

・[実践Julia入門（後藤 俊介）](https://www.amazon.co.jp/%E5%AE%9F%E8%B7%B5Julia%E5%85%A5%E9%96%80-%E5%BE%8C%E8%97%A4-%E4%BF%8A%E4%BB%8B/dp/4297133504)


・[Juliaではじめる数値計算入門（永井 佑紀）](https://www.amazon.co.jp/Julia%E3%81%A7%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B%E6%95%B0%E5%80%A4%E8%A8%88%E7%AE%97%E5%85%A5%E9%96%80-%E6%B0%B8%E4%BA%95-%E4%BD%91%E7%B4%80/dp/4297141280/ref=sr_1_2?crid=11ET9OCCLYC7O&dib=eyJ2IjoiMSJ9.jNMNj7beYNKSYpScrK5aM8WdqV2PyM5VX16cAsUO6pRjpwE6F5zsFlXAG2jGKDEGaAtqh-GgnwAWtbvJEjWdClElwOE5R7JKne1uZsD1VYeKPpYOZCYYWaPkL_MakWaa6fiApZqXWHRcoKRmpqvEm7nTJQvlJu5FUGptydniNtTbJXx-Kgarq_j21UQvl0Mg.SBxh6_FhvW_OIqbCPIG-KjeMUhPBVDsNbfMAVB7scyw&dib_tag=se&keywords=julia+%E6%95%B0%E5%80%A4%E8%A8%88%E7%AE%97&qid=1744521585&s=books&sprefix=julia+%E6%95%B0%E5%80%A4%E8%A8%88%E7%AE%97%2Cstripbooks%2C148&sr=1-2)


・[スタンフォード　ベクトル・行列からはじめる最適化数学](https://www.amazon.co.jp/%E3%82%B9%E3%82%BF%E3%83%B3%E3%83%95%E3%82%A9%E3%83%BC%E3%83%89-%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E3%83%BB%E8%A1%8C%E5%88%97%E3%81%8B%E3%82%89%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B%E6%9C%80%E9%81%A9%E5%8C%96%E6%95%B0%E5%AD%A6-%EF%BC%AB%EF%BC%B3%E6%83%85%E5%A0%B1%E7%A7%91%E5%AD%A6%E5%B0%82%E9%96%80%E6%9B%B8-%E3%82%B9%E3%83%86%E3%83%95%E3%82%A1%E3%83%B3%E3%83%BB%E3%83%9C%E3%82%A4%E3%83%89-ebook/dp/B0967Y28B6/ref=sr_1_1?crid=2GDKNFUSIEETW&dib=eyJ2IjoiMSJ9.9HxTeLNcae8dMWnlMi2pTV58tkN_LhBqTi0THYnNh19ssNZP5txa4CJ0x743w555KohaDiyQjIaEL_WE1fEfNGOuBS39e9DhGr1T9r15UpRrosJxBhSf4vXNhQKhTPfM.5GpOY-5gHtQ-BeYzaegURuquWvDCvXzIgNf_iFzYlyI&dib_tag=se&keywords=%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E8%A1%8C%E5%88%97%E3%81%8B%E3%82%89%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B+%E6%9C%80%E9%81%A9%E5%8C%96%E6%95%B0%E5%AD%A6&qid=1744521708&s=books&sprefix=%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E8%A1%8C%E5%88%97%E3%81%8B%E3%82%89%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B+%E6%9C%80%E9%81%A9%E5%8C%96%E6%95%B0%E5%AD%A6%2Cstripbooks%2C141&sr=1-1)


